In [1]:
import os
import glob
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from utility.utility import *
from utility.algorithm import *
from utility.utility import find_period_in_index, stacking
from tqdm import tqdm

In [2]:
# to get data from pulsar 2
def import_files(path, dtype):
    '''
    Import files in the syntax that I used in the notebook. That is, choose the path that contains
    all the files, then obtain all dat and inf files as dictionaries that can be accessed by filename. 
    '''

    filename = "*.dat"
    files =  [ p for p in Path(path).glob(filename) ]
    file_dict = dict()
    for file in files:
        with open(file, 'rb') as f:
            # key of the dict is the filename, value the data.
            file_dict[os.path.basename(file)] = np.fromfile(f, np.float32)

    metadata_dict = dict()
    filename = "*.inf"
    files =  [ p for p in Path(path).glob(filename) ]
    for file in files:
        with open(file) as f:
            metadata_dict[os.path.basename(file)] = f.readlines()

    return file_dict, metadata_dict 


data_dict, metadata_dict = import_files(path="../../LOFAR_2e_jaars_RP_2022_voor_studenten/Data_Pulsar2", dtype="float32")
nbins_dict = dict()
dt_dict = dict()
mjd_dict = dict()
for key, val in metadata_dict.items():
    nbins_dict[key] = int(get_metadata_item(metadata_dict[key], 'Number of bins in the time series'))
    dt_dict[key] = float(get_metadata_item(metadata_dict[key], 'Width of each time series bin (sec)'))
    mjd_dict[key] = float(get_metadata_item(metadata_dict[key], 'Epoch of observation (MJD)'))

In [3]:
# to get data from pular 1
dtype = np.dtype('float32')
path = "../../LOFAR_2e_jaars_RP_2022_voor_studenten/Data_Pulsar1/L197621_SAP0_BEAM1_DM12.44_mask.dat"
with open( path, 'rb') as f:
    pulsar1_data = np.fromfile(f, dtype)

In [4]:
# use fitresults mathematica to make function 
def sine_fit(x):
    B = 0.00166275
    A = 7.66933e-8
    omega = 39.2704
    phi = 0.345537
    y = B + A*np.sin(omega*x + phi)
    return y

#create x and y values
x_range = [number for number in np.arange(0.20+ 5.63e4, 0.50 + 5.63e4, 0.001)]
fit = [sine_fit(number) for number in x_range]


# errors from fit
B_err = 1.94857e-9
A_err = 2.62111e-9
omega_err = 1.05838
phi_err = 59587.

In [5]:
%matplotlib qt
# working data from mathematica that is from python
x_list = [56300.4688155539, 56300.45492639893, 56300.44103724313, 56300.42714808649, 56300.413258929024, 56300.39936977075, 56300.38548061164, 56300.37159145176, 56300.35770229113, 56300.34381312978]
y_list = [0.0016627957592090326, 0.00166282563896264, 0.0016628302415629193, 0.0016628131097909669, 0.0016627781654799023, 0.0016627323139534107, 0.0016626925155149316, 0.0016626750457100386, 0.0016626881480292916, 0.001662710837899844]
yerr_list = [2.353097663263867e-9, 2.3435228659575588e-9, 2.2291472249414805e-9, 1.9880488969239098e-9, 3.1889485827440945e-9, 3.46808484243367e-9, 2.1726783415015955e-9, 5e-9, 2.2821954279209845e-9, 3.615634992325844e-9]

list_MJD_All =  list( mjd_dict.values() )
# lines datapoints 
ymin = 8.4e-7 + 1.662e-3
ymax = 6.6e-7 + 1.662e-3
plt.clf()
plt.vlines(list_MJD_All, ymin, ymax, colors='black', linestyles='dotted')
plt.plot(x_list,y_list, 'o')
plt.plot(x_range, fit, color='magenta')
plt.errorbar(x_list, y_list, yerr_list, color='red', fmt='o')

print(sorted(list_MJD_All))

[56299.364628335425, 56299.61949397883, 56300.32992396778, 56300.34381312978, 56300.35770229113, 56300.37159145176, 56300.38548061164, 56300.39936977075, 56300.413258929024, 56300.42714808649, 56300.44103724313, 56300.45492639893, 56300.4688155539, 56300.48270470806]


In [13]:
# information pulsar 1 dataset
metadata_path = '../../LOFAR_2e_jaars_RP_2022_voor_studenten/Data_Pulsar1/L197621_SAP0_BEAM1_DM12.44_Mask.inf'
with open(metadata_path) as f:
    metadata = f.readlines()

period_pulsar1 = 1.3377139183200188
nbins_pulsar1= int(get_metadata_item(metadata, 'Number of bins in the time series'))
dt_pulsar1 = float(get_metadata_item(metadata, 'Width of each time series bin (sec)'))

#stacking
period_in_index_pulsar1 = find_period_in_index(period=period_pulsar1, dt=dt_pulsar1)
stack_pulsar1 = stacking(pulsar_data=pulsar1_data, period_in_index=period_in_index_pulsar1, nbins=nbins_pulsar1)

# time for plot
time_pulsar1 = []
for t in tqdm(range(len(stack_pulsar1))):
    time = dt_pulsar1*t
    time_pulsar1.append(time)


# information pulsar 2 datasets
data_pulsar2_set3 = data_dict['L83080_SAP0_BEAM0_DM39.66.dat']
x_value = mjd_dict['L83080_SAP0_BEAM0_DM39.66.inf']

period_pulsar2_set3 = sine_fit(x_value) # found with desmos
dt_pulsar2 = 2.04799998755334e-05
nbins_pulsar2 = 8800000

#stacking
period_in_index_pulsar2 = find_period_in_index(period=period_pulsar2_set3, dt=dt_pulsar2)
stack_pulsar2 = stacking(pulsar_data=data_pulsar2_set3, period_in_index=period_in_index_pulsar2, nbins=nbins_pulsar2)

#time for plot
time_pulsar2 = []
for t in tqdm(range(len(stack_pulsar2))):
    time = dt_pulsar2*t
    time_pulsar2.append(time)


100%|██████████| 81/81 [00:00<00:00, 291371.03it/s]


In [14]:
plt.plot(time_pulsar1, stack_pulsar1 / max( stack_pulsar1 ) )#, color='magenta')
plt.xlabel('Time (s)')
plt.ylabel('Relative amplitude')

Text(0, 0.5, 'Relative amplitude')

In [15]:
plt.plot(time_pulsar2, stack_pulsar2 / max( stack_pulsar2 ) )#, color='magenta')
plt.xlabel('Time (s)')
plt.ylabel('Relative amplitude')

Text(0, 0.5, 'Relative amplitude')